In [1]:
import os
os.chdir('..')

In [2]:
import torch
import scvi
import numpy as np
import pandas as pd
import scanpy as sc
import matplotlib.pyplot as plt

sc.set_figure_params(dpi=100, frameon=False, color_map='Reds', facecolor=None)
sc.logging.print_header()

/home/icb/yuge.ji/miniconda3/envs/test_env3/lib/python3.10/site-packages/torch/utils/tensorboard/__init__.py:2: DeprecationWarning: The distutils package is deprecated and slated for removal in Python 3.12. Use setuptools or check PEP 632 for potential alternatives
  from distutils.version import LooseVersion
Global seed set to 0
/home/icb/yuge.ji/miniconda3/envs/test_env3/lib/python3.10/site-packages/scanpy/_settings.py:447: DeprecationWarning: `set_matplotlib_formats` is deprecated since IPython 7.23, directly use `matplotlib_inline.backend_inline.set_matplotlib_formats()`
  IPython.display.set_matplotlib_formats(*ipython_format)


scanpy==1.9.1 anndata==0.8.0 umap==0.5.3 numpy==1.22.4 scipy==1.9.1 pandas==1.4.4 scikit-learn==1.1.2 statsmodels==0.13.2 pynndescent==0.5.7


/home/icb/yuge.ji/miniconda3/envs/test_env3/lib/python3.10/site-packages/umap/__init__.py:9: ImportWarning: Tensorflow not installed; ParametricUMAP will be unavailable
  warn(


## load model

In [3]:
hlca_path = 'datasets/hlca.h5ad'
adata = sc.read(hlca_path)
adata

AnnData object with n_obs × n_vars = 584944 × 2000
    obs: 'sample', 'original_celltype_ann', 'study_long', 'study', 'last_author_PI', 'subject_ID', 'subject_ID_as_published', 'pre_or_postnatal', 'age_in_years', 'age_range', 'sex', 'ethnicity', 'mixed_ethnicity', 'smoking_status', 'smoking_history', 'BMI', 'known_lung_disease', 'condition', 'subject_type', 'cause_of_death', 'sample_type', 'anatomical_region_coarse', 'anatomical_region_detailed', 'tissue_dissociation_protocol', 'cells_or_nuclei', 'single_cell_platform', "3'_or_5'", 'enrichment', 'sequencing_platform', 'reference_genome_coarse', 'ensembl_release_reference_genome', 'cell_ranger_version', 'disease_status', 'fresh_or_frozen', 'cultured', 'cell_viability_%', 'comments', 'Processing_site', 'dataset', 'anatomical_region_level_1', 'anatomical_region_level_2', 'anatomical_region_level_3', 'anatomical_region_highest_res', 'age', 'ann_level_1', 'ann_level_2', 'ann_level_3', 'ann_level_4', 'ann_level_5', 'ann_highest_res', 'ann_ne

In [4]:
(pd.crosstab(adata.obs.dataset, adata.obs.scanvi_label)>100).sum(axis=1)

dataset
Banovich_Kropski_2020        21
Barbry_Leroy_2020            14
Jain_Misharin_2021_10Xv1      6
Jain_Misharin_2021_10Xv2      8
Krasnow_2020                 21
Lafyatis_Rojas_2019_10Xv1     5
Lafyatis_Rojas_2019_10Xv2    13
Meyer_2019                   14
Misharin_2021                19
Misharin_Budinger_2018       10
Nawijn_2021                  14
Seibold_2020_10Xv2            4
Seibold_2020_10Xv3            5
Teichmann_Meyer_2019         10
dtype: int64

In [5]:
adata.var.to_csv('datasets/hlca_hvgs.csv')

In [6]:
from scripts import hlca_markers

In [7]:
marker_dict = hlca_markers()

In [8]:
indices = []
for dataset in adata.obs.dataset.unique():
    subset = adata[adata.obs.dataset == dataset]
    for ct in subset.obs.scanvi_label.unique():
        if ct in marker_dict.keys() and subset.obs.scanvi_label.value_counts()[ct] > 100:
            subset.obs.scanvi_label.value_counts()[ct]
            indices.append(np.random.choice(subset[subset.obs.scanvi_label == ct].obs.index, 100, replace=False))
len(indices)

39

In [9]:
adata[np.array(indices).flatten()].write('datasets/hlca_subset.h5ad')